## Tests on optimizers

Three optimizers are tested on the 4-layer CNN model with 10 epochs, including:
- Adam
- SGD
- RMSprop

In [0]:
#Adam
%reset -f
import torch
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import numpy as np
from google.colab import drive
drive.mount('/content/gdrive')
def get_data(images):
    train_images = images
    temp_images = []
    
    for i in range(len(train_images)):
        temp_images.append([])
        img_idx = i
        image = np.ascontiguousarray(train_images[img_idx], dtype=np.uint8)

        ret, threshed_img = cv2.threshold(image,
                        240, 255, cv2.THRESH_BINARY)
        img, contours, her = cv2.findContours(threshed_img, cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
        
        largest_dim = {"x":0, "y":0, "w":0, "h":0}
        largest_area = 0
        for num in contours:
            x,y,w,h = cv2.boundingRect(num)
            
            temp = 0
            
            if w > h:
                temp=w
            else:
                temp=h
                
            if temp*temp > largest_area:
                largest_area = temp*temp
                largest_dim["x"] = x
                largest_dim["y"] = y
                largest_dim["w"] = w
                largest_dim["h"] = h
        

        b = threshed_img[largest_dim["y"]:largest_dim["y"]+largest_dim["h"],largest_dim["x"]:largest_dim["x"]+largest_dim["w"]]
        
        #reducing noise
        temp_images[i] = cv2.resize(threshed_img,(28,28))
    
    temp_images = np.asarray(temp_images, dtype=np.float32)
    
    
    return temp_images
  
def split_data(train_size, valid_size, test_size):
    images = pd.read_pickle('gdrive/My Drive/Colab Notebooks/ML/comp551_project_3/input/train_images.pkl')
    labels = pd.read_csv('gdrive/My Drive/Colab Notebooks/ML/comp551_project_3/input/train_labels.csv')
    return (images[:train_size], images[train_size:train_size+valid_size],images[train_size+valid_size:]),(labels[:train_size], labels[train_size:train_size+valid_size],labels[train_size+valid_size:])


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import Adam, SGD
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.advanced_activations import LeakyReLU 
from keras.preprocessing.image import ImageDataGenerator

np.random.seed(25)


#split data in train, validation, test
(X_train, X_test, final_images), (train_labels_new, test_labels_new, final_labels)=split_data(35000, 4000, 1000)

#pre processing it, resize it
X_train = get_data(X_train)
X_test = get_data(X_test)
final_images = get_data(final_images)

# do something to fit in kera model---------------------------------------------
y_train = np.array(train_labels_new["Category"].tolist(), dtype="uint8")
y_test = np.array(test_labels_new["Category"].tolist(), dtype="uint8")
final_labels = np.array(final_labels["Category"].tolist(), dtype="uint8")

print("X_train original shape", X_train.shape)
print("y_train original shape", y_train.shape)
print("X_test original shape", X_test.shape)
print("y_test original shape", y_test.shape)

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

final_images = final_images.reshape(final_images.shape[0], 28, 28, 1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
final_images = final_images.astype('float32')

#normalization
X_train/=255
X_test/=255
final_images/=255

X_train.shape

number_of_classes = 10

Y_train = np_utils.to_categorical(y_train, number_of_classes)
Y_test = np_utils.to_categorical(y_test, number_of_classes)

y_train[0], Y_train[0]
#------------------------------------------------------------------------------

#2d convolution----------------------------------------------------------------
# Three steps to Convolution
# 1. Convolution
# 2. Activation
# 3. Polling
# Repeat Steps 1,2,3 for adding more hidden layers
# 4. After that make a fully connected network
# This fully connected network gives ability to the CNN
# to classify the samples

model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(28,28,1)))
model.add(Activation('relu'))
BatchNormalization(axis=-1)
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

BatchNormalization(axis=-1)
model.add(Conv2D(64,(3, 3)))
model.add(Activation('relu'))
BatchNormalization(axis=-1)
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
# Fully connected layer

BatchNormalization()
model.add(Dense(512))
model.add(Activation('relu'))
BatchNormalization()
model.add(Dropout(0.4))
model.add(Dense(512))
model.add(Activation('relu'))
BatchNormalization()
model.add(Dropout(0.4))
model.add(Dense(10))
model.add(Activation('softmax'))



#----------------------------------------------------------------------------------------
#generate additional images

gen = ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3,
                         height_shift_range=0.08, zoom_range=0.08)
#--------------------------------------------------------------------------------
test_gen = ImageDataGenerator()

train_generator = gen.flow(X_train, Y_train, batch_size=64)
test_generator = test_gen.flow(X_test, Y_test, batch_size=64)

for learning_rate in [0.00001,0.0001,0.001,0.01,0.1]:
  model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=learning_rate), metrics=['accuracy'])
  #fit the data and valid
  history = model.fit_generator(train_generator, steps_per_epoch=35000//64, epochs=20,validation_data=test_generator, validation_steps=4000//64)
  score = model.evaluate(X_test, Y_test)
  print('Test with learning rate: ', learning_rate)
  print('Test accuracy: ', score[1])

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
X_train original shape (35000, 28, 28)
y_train original shape (35000,)
X_test original shape (4000, 28, 28)
y_test original shape (4000,)
Epoch 1/20
546/546 [==============================] - 17s 32ms/step - loss: 2.3021 - acc: 0.1041 - val_loss: 2.3011 - val_acc: 0.0978
Epoch 2/20
546/546 [==============================] - 16s 29ms/step - loss: 2.3002 - acc: 0.1085 - val_loss: 2.2993 - val_acc: 0.1164
Epoch 3/20
546/546 [==============================] - 16s 29ms/step - loss: 2.2981 - acc: 0.1185 - val_loss: 2.2958 - val_acc: 0.1377
Epoch 4/20
546/546 [==============================] - 16s 29ms/step - loss: 2.2929 - acc: 0.1372 - val_loss: 2.2847 - val_acc: 0.1657
Epoch 5/20
546/546 [==============================] - 16s 29ms/step - loss: 2.2733 - acc: 0.1601 - val_loss: 2.2403 - val_acc: 0.1707
Epoch 6/20
546/546 [==============================] - 16s 29m

In [1]:
#SGD
%reset -f
import torch
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import numpy as np
from google.colab import drive
drive.mount('/content/gdrive')
def get_data(images):
    train_images = images
    temp_images = []
    
    for i in range(len(train_images)):
        temp_images.append([])
        img_idx = i
        image = np.ascontiguousarray(train_images[img_idx], dtype=np.uint8)

        ret, threshed_img = cv2.threshold(image,
                        240, 255, cv2.THRESH_BINARY)
        img, contours, her = cv2.findContours(threshed_img, cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
        
        largest_dim = {"x":0, "y":0, "w":0, "h":0}
        largest_area = 0
        for num in contours:
            x,y,w,h = cv2.boundingRect(num)
            
            temp = 0
            
            if w > h:
                temp=w
            else:
                temp=h
                
            if temp*temp > largest_area:
                largest_area = temp*temp
                largest_dim["x"] = x
                largest_dim["y"] = y
                largest_dim["w"] = w
                largest_dim["h"] = h
        

        b = threshed_img[largest_dim["y"]:largest_dim["y"]+largest_dim["h"],largest_dim["x"]:largest_dim["x"]+largest_dim["w"]]
        
        #reducing noise
        temp_images[i] = cv2.resize(threshed_img,(28,28))
    
    temp_images = np.asarray(temp_images, dtype=np.float32)
    
    
    return temp_images
  
def split_data(train_size, valid_size, test_size):
    images = pd.read_pickle('gdrive/My Drive/Colab Notebooks/ML/comp551_project_3/input/train_images.pkl')
    labels = pd.read_csv('gdrive/My Drive/Colab Notebooks/ML/comp551_project_3/input/train_labels.csv')
    return (images[:train_size], images[train_size:train_size+valid_size],images[train_size+valid_size:]),(labels[:train_size], labels[train_size:train_size+valid_size],labels[train_size+valid_size:])


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import Adam, SGD
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.advanced_activations import LeakyReLU 
from keras.preprocessing.image import ImageDataGenerator

np.random.seed(25)

#split data in train, validation, test
(X_train, X_test, final_images), (train_labels_new, test_labels_new, final_labels)=split_data(35000, 4000, 1000)

#pre processing it, resize it
X_train = get_data(X_train)
X_test = get_data(X_test)
final_images = get_data(final_images)

# do something to fit in kera model---------------------------------------------
y_train = np.array(train_labels_new["Category"].tolist(), dtype="uint8")

y_test = np.array(test_labels_new["Category"].tolist(), dtype="uint8")

final_labels = np.array(final_labels["Category"].tolist(), dtype="uint8")

print("X_train original shape", X_train.shape)
print("y_train original shape", y_train.shape)
print("X_test original shape", X_test.shape)
print("y_test original shape", y_test.shape)

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

final_images = final_images.reshape(final_images.shape[0], 28, 28, 1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
final_images = final_images.astype('float32')

#normalization
X_train/=255
X_test/=255
final_images/=255

X_train.shape

number_of_classes = 10

Y_train = np_utils.to_categorical(y_train, number_of_classes)
Y_test = np_utils.to_categorical(y_test, number_of_classes)

y_train[0], Y_train[0]
#------------------------------------------------------------------------------

#2d convolution----------------------------------------------------------------
# Three steps to Convolution
# 1. Convolution
# 2. Activation
# 3. Polling
# Repeat Steps 1,2,3 for adding more hidden layers
# 4. After that make a fully connected network
# This fully connected network gives ability to the CNN
# to classify the samples

model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(28,28,1)))
model.add(Activation('relu'))
BatchNormalization(axis=-1)
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

BatchNormalization(axis=-1)
model.add(Conv2D(64,(3, 3)))
model.add(Activation('relu'))
BatchNormalization(axis=-1)
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
# Fully connected layer

BatchNormalization()
model.add(Dense(512))
model.add(Activation('relu'))
BatchNormalization()
model.add(Dropout(0.4))
model.add(Dense(512))
model.add(Activation('relu'))
BatchNormalization()
model.add(Dropout(0.4))
model.add(Dense(10))
model.add(Activation('softmax'))



#----------------------------------------------------------------------------------------
#generate additional images

gen = ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3,
                         height_shift_range=0.08, zoom_range=0.08)
#--------------------------------------------------------------------------------
test_gen = ImageDataGenerator()

train_generator = gen.flow(X_train, Y_train, batch_size=64)
test_generator = test_gen.flow(X_test, Y_test, batch_size=64)

for learning_rate in [0.00001,0.0001,0.001,0.01,0.1]:
  model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=learning_rate), metrics=['accuracy'])
  #fit the data and valid
  history = model.fit_generator(train_generator, steps_per_epoch=35000//64, epochs=20,validation_data=test_generator, validation_steps=4000//64)
  score = model.evaluate(X_test, Y_test)
  print('Test with learning rate: ', learning_rate)
  print('Test accuracy: ', score[1])

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Using TensorFlow backend.


X_train original shape (35000, 28, 28)
y_train original shape (35000,)
X_test original shape (4000, 28, 28)
y_test original shape (4000,)
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/20
546/546 [==============================] - 17s 32ms/step - loss: 2.3028 - acc: 0.1004 - val_loss: 2.3029 - val_acc: 0.0915
Epoch 2/20
546/546 [==============================] - 14s 25ms/step - loss: 2.3030 - acc: 0.1002 - val_loss: 2.3028 - val_acc: 0.0912
Epoch 3/20
546/546 [==============================] - 14s 25ms/step - loss: 2.3028 - acc: 0.0989 - val_loss: 2.3028 - val_acc: 0.0899
Epoch 4/20
546/546 [==============================] - 14s 26ms/step - loss: 2.3031 - acc: 0.0988 - val_loss: 2.3032 - val_acc: 0.0899
Epoch 5/20
546/546 [==============================] - 14s 25ms/step - loss: 2.3030 - acc: 0

In [2]:
#RMSProp
%reset -f
import torch
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import numpy as np
from google.colab import drive
drive.mount('/content/gdrive')
def get_data(images):
    train_images = images
    temp_images = []
    
    for i in range(len(train_images)):
        temp_images.append([])
        img_idx = i
        image = np.ascontiguousarray(train_images[img_idx], dtype=np.uint8)

        ret, threshed_img = cv2.threshold(image,
                        240, 255, cv2.THRESH_BINARY)
        img, contours, her = cv2.findContours(threshed_img, cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
        
        largest_dim = {"x":0, "y":0, "w":0, "h":0}
        largest_area = 0
        for num in contours:
            x,y,w,h = cv2.boundingRect(num)
            
            temp = 0
            
            if w > h:
                temp=w
            else:
                temp=h
                
            if temp*temp > largest_area:
                largest_area = temp*temp
                largest_dim["x"] = x
                largest_dim["y"] = y
                largest_dim["w"] = w
                largest_dim["h"] = h
        

        b = threshed_img[largest_dim["y"]:largest_dim["y"]+largest_dim["h"],largest_dim["x"]:largest_dim["x"]+largest_dim["w"]]

        #reducing noise
        temp_images[i] = cv2.resize(threshed_img,(28,28))
    
    temp_images = np.asarray(temp_images, dtype=np.float32)
    
    
    return temp_images
  
def split_data(train_size, valid_size, test_size):
    images = pd.read_pickle('gdrive/My Drive/Colab Notebooks/ML/comp551_project_3/input/train_images.pkl')
    labels = pd.read_csv('gdrive/My Drive/Colab Notebooks/ML/comp551_project_3/input/train_labels.csv')
    return (images[:train_size], images[train_size:train_size+valid_size],images[train_size+valid_size:]),(labels[:train_size], labels[train_size:train_size+valid_size],labels[train_size+valid_size:])


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import Adam, SGD, RMSprop
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.advanced_activations import LeakyReLU 
from keras.preprocessing.image import ImageDataGenerator

np.random.seed(25)

#split data in train, validation, test
(X_train, X_test, final_images), (train_labels_new, test_labels_new, final_labels)=split_data(35000, 4000, 1000)

#pre processing it, resize it
X_train = get_data(X_train)
X_test = get_data(X_test)
final_images = get_data(final_images)

# do something to fit in kera model---------------------------------------------
y_train = np.array(train_labels_new["Category"].tolist(), dtype="uint8")
y_test = np.array(test_labels_new["Category"].tolist(), dtype="uint8")
final_labels = np.array(final_labels["Category"].tolist(), dtype="uint8")

print("X_train original shape", X_train.shape)
print("y_train original shape", y_train.shape)
print("X_test original shape", X_test.shape)
print("y_test original shape", y_test.shape)

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

final_images = final_images.reshape(final_images.shape[0], 28, 28, 1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
final_images = final_images.astype('float32')

#normalization
X_train/=255
X_test/=255
final_images/=255

X_train.shape

number_of_classes = 10

Y_train = np_utils.to_categorical(y_train, number_of_classes)
Y_test = np_utils.to_categorical(y_test, number_of_classes)

y_train[0], Y_train[0]
#------------------------------------------------------------------------------

#2d convolution----------------------------------------------------------------
# Three steps to Convolution
# 1. Convolution
# 2. Activation
# 3. Polling
# Repeat Steps 1,2,3 for adding more hidden layers
# 4. After that make a fully connected network
# This fully connected network gives ability to the CNN
# to classify the samples

model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(28,28,1)))
model.add(Activation('relu'))
BatchNormalization(axis=-1)
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

BatchNormalization(axis=-1)
model.add(Conv2D(64,(3, 3)))
model.add(Activation('relu'))
BatchNormalization(axis=-1)
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
# Fully connected layer

BatchNormalization()
model.add(Dense(512))
model.add(Activation('relu'))
BatchNormalization()
model.add(Dropout(0.4))
model.add(Dense(512))
model.add(Activation('relu'))
BatchNormalization()
model.add(Dropout(0.4))
model.add(Dense(10))
model.add(Activation('softmax'))



#----------------------------------------------------------------------------------------
#generate additional images

gen = ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3,
                         height_shift_range=0.08, zoom_range=0.08)
#--------------------------------------------------------------------------------
test_gen = ImageDataGenerator()

train_generator = gen.flow(X_train, Y_train, batch_size=64)
test_generator = test_gen.flow(X_test, Y_test, batch_size=64)

for learning_rate in [0.00001,0.0001,0.001,0.01,0.1]:
  model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=learning_rate), metrics=['accuracy'])
  #fit the data and valid
  history = model.fit_generator(train_generator, steps_per_epoch=35000//64, epochs=20,validation_data=test_generator, validation_steps=4000//64)
  score = model.evaluate(X_test, Y_test)
  print('Test with learning rate: ', learning_rate)
  print('Test accuracy: ', score[1])

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
X_train original shape (35000, 28, 28)
y_train original shape (35000,)
X_test original shape (4000, 28, 28)
y_test original shape (4000,)
Epoch 1/20
546/546 [==============================] - 16s 30ms/step - loss: 2.3021 - acc: 0.1020 - val_loss: 2.3012 - val_acc: 0.0973
Epoch 2/20
546/546 [==============================] - 15s 27ms/step - loss: 2.3010 - acc: 0.1071 - val_loss: 2.2997 - val_acc: 0.1171
Epoch 3/20
546/546 [==============================] - 16s 29ms/step - loss: 2.2993 - acc: 0.1141 - val_loss: 2.2977 - val_acc: 0.1329
Epoch 4/20
546/546 [==============================] - 16s 29ms/step - loss: 2.2968 - acc: 0.1250 - val_loss: 2.2937 - val_acc: 0.1377
Epoch 5/20
546/546 [==============================] - 16s 29ms/step - loss: 2.2915 - acc: 0.1417 - val_loss: 2.2839 - val_acc: 0.1669
Epoch 6/20
546/546 [==============================] - 16s 29m